In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import gridspec
rc('text', usetex=False)
plt.rcParams.update({'font.size': 8})
import scipy
from scipy.interpolate import Rbf, interp1d, griddata
from scipy.signal import find_peaks
from scipy.misc import derivative
from scipy.integrate import solve_ivp
from scipy.optimize import least_squares
import os
import time

In [ ]:
def fun(x, gamma, alpha, n):
    # Compute growth rates on mesh
    mu = np.exp(-x)
    vel = 0.5 * (1 - np.exp(-x))
    dx = np.mean(np.diff(x))
    
    def step(t, y):
        y = y.reshape((len(x),3))
        p1 = y[:,0]
        p2 = y[:,1]
        p3 = y[:,2]

        # Finite difference spatial derivatives
        dp1dx = np.zeros_like(x)
        dp2dx = np.zeros_like(x)
        dp3dx = np.zeros_like(x)
        for i in range(1, len(x)-1):
            dp1dx[i] = (p1[i+1] - p1[i-1]) * 0.5 / dx
            dp2dx[i] = (p2[i+1] - p2[i-1]) * 0.5 / dx
            dp3dx[i] = (p3[i+1] - p3[i-1]) * 0.5 / dx
        dp1dx[-1] = (p1[-1] - p1[-2]) / dx
        dp2dx[-1] = (p2[-1] - p2[-2]) / dx
        dp3dx[-1] = (p3[-1] - p3[-2]) / dx
        dp1dx[0] = (p1[1] - p1[0]) / dx
        dp2dx[0] = (p2[1] - p2[0]) / dx
        dp3dx[0] = (p3[1] - p3[0]) / dx

        # Update protein concs
        dy = np.zeros_like(y)
        dp1dt = alpha/(1 + p3**n) - gamma*p1 - mu*p1 - vel*dp1dx
        dp2dt = alpha/(1 + p1**n) - gamma*p2 - mu*p2 - vel*dp2dx
        dp3dt = alpha/(1 + p2**n) - gamma*p3 - mu*p3 - vel*dp3dx
        dy[:,0] = dp1dt
        dy[:,1] = dp2dt
        dy[:,2] = dp3dt
        return dy.ravel()
    return step

In [ ]:
def make_kymo(alpha, gamma, n, L, nx, nt):
    x = np.linspace(0, L, nx)
    y0 = np.zeros((nx,3))
    y0[:,1] = 5
    y0 = y0.ravel()
    res = solve_ivp(fun(x, gamma, alpha, n), t_span=(0,24), y0=y0, t_eval=np.linspace(0,24,nt), method='LSODA')
    sol = res.y.reshape((nx,3,nt))
    kymo = np.zeros((nx,nt,3))
    kymo[:,:,0] = sol[:,0,:]
    kymo[:,:,1] = sol[:,1,:]
    kymo[:,:,2] = sol[:,2,:]    
    return kymo

In [ ]:
kymo = make_kymo(alpha=1e4, gamma=0.3, n=2, L=10, nx=500, nt=500)
plt.plot(kymo[:,40,:])

In [ ]:
#for t in range(100):
#    sol[t*500/L:,:,t] = np.inf
def norm_kymo(kymo):
    nkymo = np.zeros_like(kymo)
    nkymo[:,:,0] = kymo[:,:,0] / kymo[:,:,0].max()
    nkymo[:,:,1] = kymo[:,:,1] / kymo[:,:,1].max()
    nkymo[:,:,2] = kymo[:,:,2] / kymo[:,:,2].max()
    return nkymo

In [ ]:
def map_kymo(kymo):
    rkymo = np.zeros_like(kymo)
    nx,nt,_ = kymo.shape
    for t in range(nt):
        for xx in range((nx//nt*t)//2):
            rkymo[nx+xx-(nx//nt*t)//2,t,:] = kymo[xx,t,:]
    return rkymo

In [ ]:
rkymo = map_kymo(norm_kymo(kymo))
plt.imshow(rkymo, aspect='auto')

In [ ]:
def residuals(data, L, nx, nt):
    def func(x):
        alpha, gamma, n = x
        alpha = 10**alpha
        kymo = norm_kymo(make_kymo(alpha, gamma, n, L, nx, nt))
        #rkymo = map_kymo(kymo)
        residuals = data.ravel() - kymo.ravel()
        return residuals
    return func

In [ ]:
data = norm_kymo(make_kymo(1e4, 0.3, 2, 10, 200, 100))
res = least_squares(
    residuals(data, 10, 200,100), 
    [0,0.3,2], 
    #bounds=[[0,0,1],[1e6,10,10]],
    method='lm'
)

In [ ]:
a,g,n = res.x
a = 10**a
print(a, g, n)
k = make_kymo(a, g, n, 10, 200, 100)
plt.imshow(norm_kymo(k), aspect='auto')
plt.figure()
plt.imshow(norm_kymo(data), aspect='auto')

In [ ]:
from numpy.fft import fft2, fftshift
frkymo = fftshift(fft2(rkymo[:,:,0]))

In [ ]:
plt.imshow(np.absolute(frkymo[230:270,230:270]))